# CPRD Notebook:
## Evaluation of fine-tuning the pre-trained SurvivEHR-CR model on a supervised cohort study.

Cohort study:

This notebook quantifies the performance obtained when fine-tuning the pre-trained model

In [1]:
import os
from pathlib import Path
import sys
node_type = os.getenv('BB_CPU')
venv_dir = f'/rds/homes/g/gaddcz/Projects/CPRD/virtual-envTorch2.0-{node_type}'
venv_site_pkgs = Path(venv_dir) / 'lib' / f'python{sys.version_info.major}.{sys.version_info.minor}' / 'site-packages'
if venv_site_pkgs.exists():
    sys.path.insert(0, str(venv_site_pkgs))
    print(f"Added path '{venv_site_pkgs}' at start of search paths.")
else:
    print(f"Path '{venv_site_pkgs}' not found. Check that it exists and/or that it exists for node-type '{node_type}'.")

%load_ext autoreload
%autoreload 2

print(os.getcwd())

Added path '/rds/homes/g/gaddcz/Projects/CPRD/virtual-envTorch2.0-icelake/lib/python3.10/site-packages' at start of search paths.
/rds/homes/g/gaddcz/Projects/CPRD/examples/modelling/SurvivEHR/notebooks/CompetingRisk/1_Study3_MultiMorbidity


In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import logging
import wandb
import pickle
from hydra import compose, initialize
from omegaconf import OmegaConf

from FastEHR.dataloader.foundational_loader import FoundationalDataModule

from CPRD.examples.modelling.SurvivEHR.run_experiment import run
from CPRD.src.models.survival.task_heads.causal import SurvStreamGPTForCausalModelling

import time
import polars as pl
pl.Config.set_tbl_rows(10000)
import pandas as pd
pd.options.display.max_rows = 10000


# TODO: define an env variable to fix for a local hpc environment issue, this shouldn't be needed
%env SLURM_NTASKS_PER_NODE=28   

env: SLURM_NTASKS_PER_NODE=28


# Fine-tuning on full dataset
The default configuration is for pre-training. Here we modify as necesssary

Here we choose to load in the configuration for a small **pre-trained** 11.4M parameter model, named "CR_11M". We specfiy the `fine-tune` experiment type, which will lead to running the ```SupervisedExperiment```. 

We tell this experiment that we want to perform training (true by default). Additionally, we do choose to perform testing (true by default). As this is a supervised model, this tests the ability to predict the outcomes of interest. In this notebook, this is chosen to be those of the cohort study for predicting Cardiovascular Disease in a Type 2 Diabetes Mellitus population, and we add the folder containing this dataset to the configuration. 

```Note: As this is a supervised dataset, we need to tell the DataModule that the last event observed is a target and must be stripped. This is done by passing a list of targets to the configuration, overriding the null default. This lets the DataModule know that it should process batches as supervised.```

We set the number of workers to be appropriate for the number of CPUs available to reduce bottlenecking, and tell the experiment that we do not want to limit the number of testing batches. In addition, we specify where we want any checkpoints to be saved to avoid bloating the repository.

We design a new optimisation strategy for fine-tuning. Pre-training was achieved with a warmup and cosine annealing, with rates which are no appropriate for much smaller dataset sizes seen in clinical prediction models (CPMs). We here choose a simpler strategy: of ReduceOnPlateau with no warmup, increasing the number of epochs (default is 1) and reduced validation intervals, and the addition of early stopping. Additionally, as this is not a causal model we can increase the batch size. Finally, as this CPM is not trying to predict the value of any outcomes, we set the value weight to zero allowing the model to focus entirely on optimising survival outcome prediction.

In [5]:
pre_trained_model_ids = ['SurvivEHR-cr-small', 'SurvivEHR-cr-small-v1', 'SurvivEHR-cr-384', 'SurvivEHR-cr-384-v1', 'crPreTrain_small_1337']
experiments = ["mm"] 
experiment_types = [ "fine-tune-cr"] 
adapter = 8

## Get outcome list

In [9]:
if True:
    for pre_trained_model in pre_trained_model_ids[1:2]:
        print(pre_trained_model)
    
        for experiment, experiment_type in zip(experiments, experiment_types):
        
            wandb.finish()
            # load the configuration file, override any settings 
            with initialize(version_base=None, config_path="../../../confs", job_name="testing_notebook"):
                cfg = compose(config_name="config_CompetingRisk11M", 
                              overrides=[# Experiment setup
                                         f"experiment.project_name='SurvivEHR-Study3-MM'",
                                         f"experiment.type='{experiment_type}'",
                                         f"experiment.run_id='{pre_trained_model}'",
                                         f"experiment.fine_tune_id='{experiment}-{experiment_type}-A{adapter}-notebook'",
                                         "experiment.train=True",
                                         "experiment.test=True",
                                         "experiment.notes=Table result",
                                         # Dataloader
                                         "data.batch_size=128",
                                         "data.meta_information_path=/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/meta_information_QuantJenny.pickle",
                                         "data.min_workers=3",
                                         "data.global_diagnoses=True",
                                         "data.repeating_events=False",
                                         # Optimiser
                                         "optim.num_epochs=20",
                                         "optim.limit_test_batches=null",
                                         "optim.scheduler=ReduceOnPlateau",
                                         "optim.scheduler_warmup=False",
                                         "optim.learning_rate=1e-3",
                                         "optim.val_check_interval=50",
                                         "optim.early_stop=True",
                                         "optim.early_stop_patience=4",
                                         "optim.limit_val_batches=0.035",
                                         "optim.accumulate_grad_batches=4",
                                         # Model
                                         # "transformer.n_embd=384",
                                         f"transformer.use_fine_tune_adapter={False if adapter is False else True}",
                                         f"transformer.adapter_dim={8 if adapter is False else adapter}",
                                         "transformer.block_size=512", 
                                        ]
                             )
            
            match experiment.lower():
                case "mm":
                    cfg.data.path_to_ds="/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_MultiMorbidity/"

                    # Load dataset in most minimal form (this isnt used for the experiment - only to extract the token names for the diagnoses)
                    dm = FoundationalDataModule(path_to_db=cfg.data.path_to_db,
                                                path_to_ds=cfg.data.path_to_ds,
                                                overwrite_meta_information=cfg.data.meta_information_path,
                                                load=True)
                    conditions = (
                        dm.tokenizer._event_counts.filter((pl.col("COUNT") > 0) &
                            (pl.col("EVENT").str.contains(r'^[A-Z0-9_]+$')))
                          .select("EVENT")
                          .to_series()
                          .to_list()
                    )
                    cfg.experiment.fine_tune_outcomes=conditions
            
            
            model, dm = run(cfg)
            print(f"Loaded model with {sum(p.numel() for p in model.parameters())/1e6} M parameters")
            wandb.finish()



SurvivEHR-cr-small-v1


/rds/bear-apps/2022a/EL8-ice/software/PyTorch-Lightning/2.1.0-foss-2022a-CUDA-11.7.0/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:173: Found keys that are in the model state dict but not in the checkpoint: ['model.transformer.blocks.0.adapter_1.proj.0.weight', 'model.transformer.blocks.0.adapter_1.proj.0.bias', 'model.transformer.blocks.0.adapter_1.proj.2.weight', 'model.transformer.blocks.0.adapter_1.proj.2.bias', 'model.transformer.blocks.0.adapter_2.proj.0.weight', 'model.transformer.blocks.0.adapter_2.proj.0.bias', 'model.transformer.blocks.0.adapter_2.proj.2.weight', 'model.transformer.blocks.0.adapter_2.proj.2.bias', 'model.transformer.blocks.1.adapter_1.proj.0.weight', 'model.transformer.blocks.1.adapter_1.proj.0.bias', 'model.transformer.blocks.1.adapter_1.proj.2.weight', 'model.transformer.blocks.1.adapter_1.proj.2.bias', 'model.transformer.blocks.1.adapter_2.proj.0.weight', 'model.transformer.blocks.1.adapter_2.proj.0.bias', 'model.transformer.blocks.1.adapte

/rds/bear-apps/2022a/EL8-ice/software/PyTorch-Lightning/2.1.0-foss-2022a-CUDA-11.7.0/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:630: Checkpoint directory /rds/projects/s/subramaa-mum-predict/CharlesGadd_Oxford/FoundationModelOutput/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type                            | Params
---------------------------------------------------------------
0 | model      | SurvStreamGPTForCausalModelling | 11.3 M
1 | surv_layer | ODESurvCompetingRiskLayer       | 31.7 K
---------------------------------------------------------------
120 K     Trainable params
11.2 M    Non-trainable params
11.3 M    Total params
45.286    Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Testing: |          | 0/? [00:00<?, ?it/s]

                                    		 Bad sample tokens: tensor([[255,   0,   0,   0,   0]])
                                    		 and corresponding ages tensor([[10.7184,  0.0000,  0.0000,  0.0000,  0.0000]])
                                    		 Bad sample tokens: tensor([[234,   0,   0,   0,   0]])
                                    		 and corresponding ages tensor([[10.1036,  0.0000,  0.0000,  0.0000,  0.0000]])
                                    		 Bad sample tokens: tensor([[152,   0,   0,   0,   0]])
                                    		 and corresponding ages tensor([[10.2855,  0.0000,  0.0000,  0.0000,  0.0000]])
                                    		 Bad sample tokens: tensor([[217,   0,   0,   0,   0]])
                                    		 and corresponding ages tensor([[10.0384,  0.0000,  0.0000,  0.0000,  0.0000]])
                                    		 Bad sample tokens: tensor([[260,   0,   0,   0,   0]])
                                    		 and corresponding a

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃             Test metric             ┃            DataLoader 0             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  Test:OutcomePerformanceMetricsctd  │         0.7261480689048767          │
│  Test:OutcomePerformanceMetricsibs  │         0.10712722074406386         │
│ Test:OutcomePerformanceMetricsinbll │         0.3345218206705762          │
│              test_loss              │          2.73171067237854           │
└─────────────────────────────────────┴─────────────────────────────────────┘

Loaded model with 11.321432 M parameters


Scheduler,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test:OutcomePerformanceMetricsctd,▁
Test:OutcomePerformanceMetricsibs,▁
Test:OutcomePerformanceMetricsinbll,▁
Val:OutcomePerformanceMetricsctd,▁▂▃▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇██████████████████████
Val:OutcomePerformanceMetricsibs,█▆▆▅▅▄▄▄▄▄▃▃▃▃▂▃▂▂▂▃▂▂▁▁▁▁▁▂▂▁▁▁▁▂▁▁▂▁▁▁
Val:OutcomePerformanceMetricsinbll,█▇▆▆▆▅▅▅▅▄▄▃▃▄▃▄▂▃▂▃▂▂▂▁▂▁▁▂▂▁▂▁▁▂▂▁▂▁▁▁
epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
test_loss,▁
train_loss,█▇▇▄▆▄▇▅▆▄▆▆▄▅▄▆▄▂▃▄▅▄▃▄▇▅▃▃▇▁▄▄▅▆▃
trainer/global_step,▁▂▃▅▆▇██████████████████████████████████


```
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃             Test metric             ┃            DataLoader 0             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
└─────────────────────────────────────┴─────────────────────────────────────┘
```

In [5]:
# [_i if _i == _i.upper() else 0 for _i in dm.train_set.tokenizer._stoi.keys()]

# Fine-tuning on sub-set of data

In [6]:
sample_sizes = [int(np.exp(_log_n)) for _log_n in np.linspace(np.log(3000), np.log(500000), 10)]
print(sample_sizes)

[2999, 5296, 9351, 16509, 29148, 51461, 90856, 160407, 283203, 500000]


In [ ]:
sample_sizes = [int(np.exp(_log_n)) for _log_n in np.linspace(np.log(3000), np.log(500000), 10)]      # [3000, 12500, 30000, 60000, 100000]: # 600, 1200, 
sample_sizes = [None]

accumulate_grad_batches = 5


for pre_trained_model in pre_trained_model_ids[1:2]:
    print(pre_trained_model)

    for experiment, experiment_type in zip(experiments, experiment_types):

        for sample_size in sample_sizes:

            wandb.finish()
            # load the configuration file, override any settings 
            with initialize(version_base=None, config_path="../../../confs", job_name="testing_notebook"):
                cfg = compose(config_name="config_CompetingRisk11M", 
                              overrides=[# Experiment setup
                                         f"experiment.project_name='SurvivEHR-Study1-CVD'",
                                         f"experiment.type='{experiment_type}'",
                                         f"experiment.run_id='{pre_trained_model}'",
                                         f"experiment.fine_tune_id='{experiment}-{experiment_type}-A{adapter}-Ns{sample_size}-500-notebook'",
                                         "experiment.train=True",
                                         "experiment.test=True",
                                         "experiment.notes=Ablation on increasing cohort study size result",
                                         # Dataloader
                                         "data.batch_size=128",
                                         "data.meta_information_path=/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/meta_information_QuantJenny.pickle",
                                         "data.min_workers=3",
                                         "data.global_diagnoses=True",
                                         "data.repeating_events=False",
                                         # f"data.subsample_training={sample_size}",
                                         # Optimiser
                                         "optim.num_epochs=500",
                                         "optim.limit_test_batches=null",
                                         "optim.scheduler=ReduceOnPlateau",
                                         "optim.scheduler_warmup=False",
                                         "optim.learning_rate=1e-3",
                                         "optim.val_check_interval=0.25",
                                         "optim.early_stop=True",
                                         "optim.early_stop_patience=4",
                                         "optim.limit_val_batches=0.035",
                                         f"optim.accumulate_grad_batches={accumulate_grad_batches}",
                                         # Model
                                         # "transformer.n_embd=384",
                                         f"transformer.use_fine_tune_adapter={False if adapter is False else True}",
                                         f"transformer.adapter_dim={8 if adapter is False else adapter}",
                                         "transformer.block_size=500", 
                                        ]
                             )
            
            
            match experiment.lower():
                case "cvd":
                    cfg.data.path_to_ds="/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/"
                    cfg.experiment.fine_tune_outcomes=["IHDINCLUDINGMI_OPTIMALV2", "ISCHAEMICSTROKE_V2", "MINFARCTION", "STROKEUNSPECIFIED_V2", "STROKE_HAEMRGIC"]
                case "hypertension":
                    cfg.data.path_to_ds="/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/"
                    cfg.experiment.fine_tune_outcomes=["HYPERTENSION"]
            
            
            model, dm = run(cfg)
            print(f"Loaded model with {sum(p.numel() for p in model.parameters())/1e6} M parameters")
            wandb.finish()


SurvivEHR-cr-small-v1


/rds/bear-apps/2022a/EL8-ice/software/PyTorch-Lightning/2.1.0-foss-2022a-CUDA-11.7.0/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:173: Found keys that are in the model state dict but not in the checkpoint: ['model.transformer.blocks.0.adapter_1.proj.0.weight', 'model.transformer.blocks.0.adapter_1.proj.0.bias', 'model.transformer.blocks.0.adapter_1.proj.2.weight', 'model.transformer.blocks.0.adapter_1.proj.2.bias', 'model.transformer.blocks.0.adapter_2.proj.0.weight', 'model.transformer.blocks.0.adapter_2.proj.0.bias', 'model.transformer.blocks.0.adapter_2.proj.2.weight', 'model.transformer.blocks.0.adapter_2.proj.2.bias', 'model.transformer.blocks.1.adapter_1.proj.0.weight', 'model.transformer.blocks.1.adapter_1.proj.0.bias', 'model.transformer.blocks.1.adapter_1.proj.2.weight', 'model.transformer.blocks.1.adapter_1.proj.2.bias', 'model.transformer.blocks.1.adapter_2.proj.0.weight', 'model.transformer.blocks.1.adapter_2.proj.0.bias', 'model.transformer.blocks.1.adapte

/rds/bear-apps/2022a/EL8-ice/software/PyTorch-Lightning/2.1.0-foss-2022a-CUDA-11.7.0/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:630: Checkpoint directory /rds/projects/s/subramaa-mum-predict/CharlesGadd_Oxford/FoundationModelOutput/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type                            | Params
---------------------------------------------------------------
0 | model      | SurvStreamGPTForCausalModelling | 11.3 M
1 | surv_layer | ODESurvCompetingRiskLayer       | 27.1 K
---------------------------------------------------------------
115 K     Trainable params
11.2 M    Non-trainable params
11.3 M    Total params
45.268    Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

In [ ]:
wandb.finish()

In [ ]:
dm.tokenizer._event_counts

In [ ]:
all_CVD = {
    "SurvivEHR-cr-small-v1": (
        "SurvivEHR-CR",
        [2999, 5296, 9351, 16509, 29148, 51461, 90856, 160407, 283203, 500000]],
        [0.6270919442176819],
        [0.03389651543792928],
        [0.14657540914939907]
        ),
    
}

